In [ ]:
import MeCab
import os.path, json,random

dic={}
dic_name = 'dic_data'




    
def make_dic(wordlist):
    global dic
    w1 = ""
    w2 = ""
    if len(wordlist) < 3:
        return
    
    for w in wordlist:
        word = w[0]
        if word == "" or word == "\r\n" or word == "\n": continue
        if word == "。" or word == "?" or word == "？":
            w1 = ""
            w2 = ""
            continue
        
        if w1 and w2:
            for w in wordlist:
                if w1 not in dic: dic[w1]={}
                if w2 not in dic[w1]: dic[w1][w2]={}
                if word not in dic[w1][w2]: dic[w1][w2][word] = 0
            dic[w1][w2][word] += 1
        
        w1, w2 = w2, word
    
    json.dump(dic, open(dic_name, 'w', encoding="utf-8"))
    
def response(tx):
    word = []
    w1 = tx[0]
    word.append(w1)
    key = dic[w1].keys()
    w2 = random.choice(list(key))
    word.append(w2)
    
    while True:
        if w1 in dic and w2 in dic[w1]:
            w3key = dic[w1][w2].keys()
            w3 = random.choice(list(w3key))
        else:
            w3 = ""
        word.append(w3)
        if w3 == "" or w3 =="。" or w3 =="?" or w3 =="？":break
       
            
        w1, w2 = w2 , w3
        
    return "".join(word)
        
if os.path.exists(dic_name): 
    dic = json.load(open(dic_name, 'r'))


while True:
    text = input("You>")
    tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")
    tagger.parse("")
    word = tagger.parseToNode(text)
    
    wordlist = []
    while word is not None:
        fea = word.feature.split(",")[0]
        if fea != ["BOS/EOS"]:
            sur = word.surface
            wordlist.append([sur, fea])
            word = word.next
        
    make_dic(wordlist)
    
    for tx in wordlist:
        if tx[1] in ["感動詞"]:
            print("Bot>"+tx[0])
            break
        if tx[1] in ["名詞","動詞", "形容詞"]:
            print("Bot>" +  response(tx))
            break
            
    

You>猫が好き。
Bot>猫が好き
You>猫
Bot>猫が好き
You>犬と遊ぶ。
Bot>犬と遊ぶ
You>犬と走る。
Bot>犬と走る
You>犬
Bot>犬と遊ぶ
You>犬
Bot>犬と走る
